In [2]:
import pandas as pd
import geopandas as gpd
import requests,json
from shapely import geometry, ops
import numpy as np
pd.set_option('display.max_columns', None)

In [3]:
cells=pd.read_csv('valid_cells.csv')
cells['index']=cells.index
cells=cells.sort_values(by='index')
cells['cell_to']=cells.cell_id.apply(lambda x: "cell_id_"+str(x))

In [4]:
cells

,Unnamed: 0,cell_id,cell_x,cell_y,center_lat,center_lng,center_stateplane_x,center_stateplane_y,index,cell_to
0,0,23,23,0,35.974934,-86.634428,542782.7388,182319.7068,0,cell_id_23
1,1,24,24,0,35.975026,-86.616584,544392.0788,182319.7068,1,cell_id_24
2,2,51,21,1,35.989245,-86.670238,539564.0588,183929.0468,2,cell_id_51
3,3,52,22,1,35.989343,-86.652391,541173.3988,183929.0468,3,cell_id_52
4,4,53,23,1,35.989438,-86.634543,542782.7388,183929.0468,4,cell_id_53
...,...,...,...,...,...,...,...,...,...,...
520,520,855,15,28,36.380194,-86.781166,529908.0188,227381.2268,520,cell_id_855
521,521,856,16,28,36.380309,-86.763231,531517.3588,227381.2268,521,cell_id_856
522,522,857,17,28,36.380421,-86.745295,533126.6988,227381.2268,522,cell_id_857
523,523,886,16,29,36.396501,-86.763387,531517.3588,229178.0402,523,cell_id_886


In [3]:
begin='http://127.0.0.1:5000/table/v1/driving/'
cells['combined']=cells.center_lng.apply(str)+','+cells.center_lat.apply(str)
sep=';'
url=begin+sep.join(cells['combined'].tolist()).strip(';')

In [4]:
r=requests.get(url)
data2=r.json()
durations=np.array(data2['durations'])

In [32]:
def calculate_bearing(lat1, lng1, lat2, lng2):
    """
    Calculate the compass bearing(s) between pairs of lat-lng points.
    Vectorized function to calculate (initial) bearings between two points'
    coordinates or between arrays of points' coordinates. Expects coordinates
    in decimal degrees. Bearing represents angle in degrees (clockwise)
    between north and the geodesic line from point 1 to point 2.
    Parameters
    ----------
    lat1 : float or numpy.array of float
        first point's latitude coordinate
    lng1 : float or numpy.array of float
        first point's longitude coordinate
    lat2 : float or numpy.array of float
        second point's latitude coordinate
    lng2 : float or numpy.array of float
        second point's longitude coordinate
    Returns
    -------
    bearing : float or numpy.array of float
        the bearing(s) in decimal degrees
    """
    # get the latitudes and the difference in longitudes, in radians
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    d_lng = np.radians(lng2 - lng1)

    # calculate initial bearing from -180 degrees to +180 degrees
    y = np.sin(d_lng) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_lng)
    initial_bearing = np.degrees(np.arctan2(y, x))

    # normalize to 0-360 degrees to get compass bearing
    return initial_bearing % 360

In [5]:
df = pd.DataFrame(durations, columns = cells.cell_to)
df['cell_from']=cells.cell_to
df=df.set_index('cell_from')
df.to_parquet('traveltime.parquet')

In [12]:
df=gpd.read_file('USA_Tennessee.geojson')
davidson=df[df.County=='DAVIDSON'].copy()
osm=pd.read_csv('USA_Tennessee.csv')
davidson=davidson.merge(osm,on='XDSegID',how='left')
davidson['osm_way']=davidson.OSMWayIDs.apply(lambda x: x.split(';'))
davidson=davidson.drop(['OSMWayIDs', 'OSMWayDirections', 'WayStartOffset_m', 'WayEndOffset_m',
       'WayStartOffset_percent', 'WayEndOffset_percent','AllTmcList','PrimaryTmc','RoadNumber','Country','State'],axis=1)
davidson=davidson.explode('osm_way')
davidson = davidson[['XDSegID','osm_way','Bearing','geometry']]
davidson.columns= davidson.columns.str.strip().str.lower()
davidson.to_pickle('davidson_osm_inrix.pkl')

In [45]:
davidson['bearings']=davidson.apply(lambda x: calculate_bearing(x.geometry.coords.xy[1][0], x.geometry.coords.xy[0][0], x.geometry.coords.xy[1][-1], x.geometry.coords.xy[0][-1]),axis=1)

In [43]:
davidson.geometry.iloc[0].coords.xy[1][-1]

36.11978729537523

In [48]:
davidson.to_pickle('segment_bearing.pkl')

In [50]:
davidsonedges=pd.read_pickle('nashville_edges.pkl')

In [52]:
davidson['osm_way']=davidson.osm_way.apply(int)

In [56]:
davidson=davidson.rename(columns={'osm_way':'osmid'})

In [62]:
edges=davidsonedges.merge(davidson,on='osmid')

In [63]:
edges=edges.drop(['travel_time', 'name', 'bridge', 'maxspeed',
       'tunnel', 'junction', 'access', 'width', 'service', 'area',
       'geometry_x','geometry_y'],axis=1)
edges

,u,v,key,osmid,oneway,lanes,ref,highway,length,speed_kph,bearing_x,xdsegid,bearing_y,bearings
0,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431087,E,122.129474
1,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431111,W,302.134457
2,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431137,E,159.536146
3,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431162,W,339.539215
4,202403383,7018685948,0,788566457,False,NaN,TN 96,primary,21.084,76,355.791817,1524431087,E,122.129474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492599,9071903931,4053750012,0,978323894,True,4,TN 254,primary,11.368,56,49.051447,450431085,E,48.481656
492600,9052743020,202514092,0,978349168,True,5,US 70S,primary,32.507,76,218.148961,1524271231,W,218.919635
492601,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524450762,E,71.349009
492602,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524526824,E,82.603862


In [65]:
edges['bearing_diff']=edges.apply(lambda x: abs(x.bearing_x-x.bearings) >=90,axis=1)

In [66]:
edges

,u,v,key,osmid,oneway,lanes,ref,highway,length,speed_kph,bearing_x,xdsegid,bearing_y,bearings,bearing_diff
0,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431087,E,122.129474,False
1,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431111,W,302.134457,True
2,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431137,E,159.536146,False
3,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431162,W,339.539215,True
4,202403383,7018685948,0,788566457,False,NaN,TN 96,primary,21.084,76,355.791817,1524431087,E,122.129474,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492599,9071903931,4053750012,0,978323894,True,4,TN 254,primary,11.368,56,49.051447,450431085,E,48.481656,False
492600,9052743020,202514092,0,978349168,True,5,US 70S,primary,32.507,76,218.148961,1524271231,W,218.919635,False
492601,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524450762,E,71.349009,False
492602,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524526824,E,82.603862,False


In [67]:
edges=edges.drop(edges[edges.bearing_diff].index)

In [70]:
edges=edges.rename(columns={'bearing_x':'bearing_osm','bearings':'bearing_inrix','bearing_y':'osm_direction'})

In [72]:
edges=edges.drop('bearing_diff',axis=1)

In [76]:
edges.to_parquet('nashville_osm_edges.parquet')

In [75]:
edges

,u,v,key,osmid,oneway,lanes,ref,highway,length,speed_kph,bearing_osm,xdsegid,osm_direction,bearing_inrix
0,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431087,E,122.129474
2,202403383,202403384,0,788566457,False,NaN,TN 96,primary,104.314,76,173.036265,1524431137,E,159.536146
5,202403383,7018685948,0,788566457,False,NaN,TN 96,primary,21.084,76,355.791817,1524431111,W,302.134457
7,202403383,7018685948,0,788566457,False,NaN,TN 96,primary,21.084,76,355.791817,1524431162,W,339.539215
8,202403384,7301200108,0,788566457,False,NaN,TN 96,primary,28.866,76,168.076775,1524431087,E,122.129474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492599,9071903931,4053750012,0,978323894,True,4,TN 254,primary,11.368,56,49.051447,450431085,E,48.481656
492600,9052743020,202514092,0,978349168,True,5,US 70S,primary,32.507,76,218.148961,1524271231,W,218.919635
492601,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524450762,E,71.349009
492602,9053403745,202180056,0,978416859,True,3,I 440,motorway,87.395,109,80.050141,1524526824,E,82.603862


In [113]:
davidsonmodel=pd.read_parquet('davidson_inrix_model_month_day_hour.parquet')

In [114]:
davidsonmodel['speed_mean']=davidsonmodel.speed_mean.apply(lambda x: int(x* 1.69093))

In [115]:
davidsonmodel

,xdsegid,hour,day,month,speed_mean,reference_speed
309,396068455,0,mon,1,27,12.0
3552,156232700,0,mon,1,36,21.0
3664,1524596717,0,mon,1,64,37.0
5794,1524628176,0,mon,1,76,44.0
7441,450430870,0,mon,1,60,34.0
...,...,...,...,...,...,...
14969866,159981790,23,sun,12,18,10.0
14969978,160139086,23,sun,12,38,23.0
14970055,1524484032,23,sun,12,80,43.0
14970121,449617541,23,sun,12,37,20.0


In [97]:
datapoints=davidsonmodel[['day','hour','month']].drop_duplicates()

In [98]:
datapoints

,day,hour,month
309,mon,0,1
306,mon,1,1
458,mon,2,1
2158,mon,3,1
5086,mon,4,1
...,...,...,...
756048,sun,19,12
769283,sun,20,12
755993,sun,21,12
769247,sun,22,12


In [132]:
for row in datapoints.itertuples():
    samplemodel=davidsonmodel[(davidsonmodel.day==row.day) & (davidsonmodel.month==row.month) & (davidsonmodel.hour==row.hour)]
    edges2=edges.merge(samplemodel,on='xdsegid',how='left')
    edges2['speed_kph']=edges2.apply(lambda x: x.speed_mean if not np.isnan(x.speed_mean) else x.speed_kph,axis=1)
    edges2=edges2.groupby(['u','v','key'],as_index=False).agg({'speed_kph':'mean'})
    edges2['speed_kph']=edges2.speed_kph.apply(int)
    edges2[['u','v','speed_kph']].to_csv(f'segment_speed_data/segment_speed_data_{row.month}_{row.day}_{row.hour}.csv',index=False,header=False)


In [128]:
edges2

,u,v,key,speed_kph
0,37060248,37060265,0,114
1,37060256,1236641346,0,114
2,37060265,7875035064,0,114
3,37060270,37060256,0,114
4,37060277,37060311,0,114
...,...,...,...,...
81184,9184382514,9186304456,0,64
81185,9186304332,202289122,0,63
81186,9186304332,6995662839,0,64
81187,9186304456,202289122,0,64


In [137]:
!ls -lhrt segment_speed_data/*.parquet

-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:39 segment_speed_data/traveltime_1_mon_0.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:39 segment_speed_data/traveltime_1_mon_1.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:40 segment_speed_data/traveltime_1_mon_2.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:40 segment_speed_data/traveltime_1_mon_3.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:40 segment_speed_data/traveltime_1_mon_4.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:48 segment_speed_data/traveltime_1_mon_5.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:48 segment_speed_data/traveltime_1_mon_6.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:49 segment_speed_data/traveltime_1_mon_7.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:49 segment_speed_data/traveltime_1_mon_8.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct 21 16:49 segment_speed_data/traveltime_1_mon_9.parquet
-rw-r--r--  1 abhishek  staff   1.8M Oct